In [2]:
import pandas as pd
import json
import os
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import wasserstein_distance
# sns.set(style='ticks', context='paper', font='CMU Sans Serif')
sns.set(style='ticks', context='paper')
from collections import defaultdict
import numpy as np
from util import plot_cdf

In [3]:
save_path="/data2/lichenni/ns3"
file2matrix = {
        '../../workload/spatials/cluster_a.json': 'Traffic Matrix A',
        '../../workload/spatials/cluster_b.json': 'Traffic Matrix B',
        '../../workload/spatials/cluster_c.json': 'Traffic Matrix C',
    }
file2oversub = {
        'spec/cluster_1_to_1.json': '1-to-1',
        'spec/cluster_2_to_1.json': '2-to-1',
        'spec/cluster_4_to_1.json': '4-to-1',
    }

P99_PERCENTILE_LIST = np.arange(1, 101, 1)

MTU=1000
BDP = 10 * MTU
bin_size_list=[MTU, BDP, 5 * BDP]
labels = {0: '0<size<=MTU', 1:'MTU<size<=BDP', 2:'BDP<size<=5BDP', 3:'5BDP<size'}

n_size_bucket_list_output=len(bin_size_list)+1
n_percentiles=len(P99_PERCENTILE_LIST)

In [1]:
# Given one mix, get the relationship between the flow size buckets and the slowdown estiamtion error
N_FLOWS=500*100*4
N_FLOWS_PER_PATH=400
N_FLOW_THRESHOLD = [1, 1, 1, 1]
worst_low_id=2
# mlsys_dir="mlsys_0114_const_bt10"
mlsys_dir="mlsys_final_reprod_v0"
# mlsys_dir="mlsys_bdp_bt1"
mix_dir = f'../data/{worst_low_id}'
# Accuracy metrics
df_ns3 = pd.read_csv(f'{mix_dir}/ns3/records.csv')
df_pmn_m = pd.read_csv(f'{mix_dir}/pmn-m/records.csv')
df_mlsys = [[] for _ in range(n_size_bucket_list_output)]

n_freq_list=[]
n_flow_list=[]
sizes=df_pmn_m['size']

# mix_dir = f'../data_test/{worst_low_id}'
path_idx=0
while os.path.exists(f'{mix_dir}/{mlsys_dir}/{path_idx}/fct_mlsys.txt'):
    with open(f'{mix_dir}/{mlsys_dir}/path_{path_idx}.txt', 'r') as file:
        lines = file.readlines()
        data=lines[0].strip().split(",")
        n_freq_list.append(int(data[-1]))
        
        flowid_list=[int(tmp) for tmp in lines[2].strip().split(",")]
        size_list=[sizes[flowid] for flowid in flowid_list]
        
        n_links=len(data[0].split("|"))-1
        tmp=np.digitize(size_list, bin_size_list)
        # tmp=np.digitize(size_list, bin_size_dict[n_links])
        # Count occurrences of each bin index
        bin_counts = np.zeros(n_size_bucket_list_output)
        for bin_idx in tmp:
            bin_counts[bin_idx]+=1
        # n_flow_list.append(bin_counts>=N_FLOW_THRESHOLD)
        n_flow_list.append(bin_counts)
    path_idx+=1
n_flow_list=np.array(n_flow_list)
# print("n_flow_list: ",n_flow_list.shape)
n_flow_list_sum=n_flow_list.sum(axis=0)
# print("n_flow_list_sum: ",n_flow_list_sum)

with open(f'{mix_dir}/{mlsys_dir}/path.txt', 'r') as file:
    lines = file.readlines()
    lines=lines[1:]
    for line_idx,line in enumerate(lines):
        data=line.strip().split(",")
        data = [float(value) for value in data]
        n_freq=n_freq_list[line_idx//n_size_bucket_list_output]
        
        # prop_tmp=n_flow_list[line_idx//n_size_bucket_list_output]/np.sum(n_flow_list[line_idx//n_size_bucket_list_output])
        # num_tmp=int(N_FLOWS_PER_PATH*prop_tmp[line_idx%n_size_bucket_list_output])
        # data_sampled=np.random.choice(data,num_tmp,replace=True)
        # for _ in range(n_freq):
        #     df_mlsys[line_idx%n_size_bucket_list_output].extend(data_sampled)
        if n_flow_list[line_idx//n_size_bucket_list_output][line_idx%n_size_bucket_list_output]>=N_FLOW_THRESHOLD[line_idx%n_size_bucket_list_output]:
            for _ in range(n_freq):
                df_mlsys[line_idx%n_size_bucket_list_output].extend(data)
        # else:
        #     print("skip: ",data[0],data[-1])
df_mlsys_shape=[len(df_mlsys[i]) for i in range(len(df_mlsys))]

sizes=df_ns3['size']
bin=np.digitize(sizes, bin_size_list)

bin_counts = np.bincount(bin)
total_count = np.sum(bin_counts)
# # Calculate the ratio for each bucket
bucket_ratios = bin_counts / total_count
print("bucket_ratios: ",bucket_ratios)
bucket_ratios_sampled=n_flow_list_sum/sum(n_flow_list_sum)
print("bucket_ratios_sampled: ",bucket_ratios_sampled)
            
df_mlsys_bucket=np.array([np.percentile(df_mlsys[i],99) for i in range(len(df_mlsys))])
print("df_mlsys_bucket: ",df_mlsys_bucket)
# df_mlsys_p99=np.sum(np.multiply(df_mlsys_bucket.T, bucket_ratios_sampled).T,axis=0)
df_mlsys_total=[]
for i in range(len(df_mlsys)):
    # df_mlsys_total.extend(df_mlsys[i])
    n_tmp=int(N_FLOWS*bucket_ratios_sampled[i])
    df_mlsys_total.extend(np.random.choice(df_mlsys[i],n_tmp,replace=True))
df_mlsys_p99=np.percentile(df_mlsys_total,99)

sldn_ns3=df_ns3['slowdown']
sldn_pmn_m=df_pmn_m['slowdown']
sldn_ns3_p99=np.percentile(sldn_ns3,99)
sldn_pmn_m_p99=np.percentile(sldn_pmn_m,99)

print("sldn_ns3: ",sldn_ns3_p99," sldn_pmn_m: ", sldn_pmn_m_p99," df_mlsys: ", df_mlsys_p99)
error=(sldn_pmn_m_p99-sldn_ns3_p99)/sldn_ns3_p99
error_mlsys=(df_mlsys_p99-sldn_ns3_p99)/sldn_ns3_p99

# assert df_ns3.shape[0]==df_pmn_m.shape[0]==df_ns3_path.shape[0]==sldn_flowsim.shape[0]
print(f"df_ns3: {df_ns3.shape[0]}, df_pmn_m: {df_pmn_m.shape[0]}, df_mlsys: {df_mlsys_shape}")
# assert df_ns3.shape[0]==df_pmn_m.shape[0]

NameError: name 'pd' is not defined

In [ ]:
_fontsize=15
plt.figure(0,figsize=(12, 6))
plt.suptitle(f"Slowdown CDF for mix-{worst_low_id} in {mlsys_dir}\npmn_error={error}, mlsys_error={error_mlsys}",fontsize=_fontsize+5)
plt.rcParams['legend.fontsize'] = _fontsize
for i in range(len(labels)):
    tmp_sldn_ns3 = np.extract(bin==i, sldn_ns3)
    tmp_sldn_pmn_m = np.extract(bin==i, sldn_pmn_m)
    tmp_sldn_mlsys=df_mlsys[i]
    print(f"{i}: ", min(tmp_sldn_mlsys))
    plt.subplot(2, 2, i + 1)
    plt.title(f"{labels[i]}-#{len(tmp_sldn_ns3)}",fontsize=_fontsize)
    plt.plot(np.sort(tmp_sldn_ns3), (np.arange(len(tmp_sldn_ns3))/len(tmp_sldn_ns3)*100), label='ns3', color='blue', linewidth=3)
    plt.plot(np.sort(tmp_sldn_pmn_m), (np.arange(len(tmp_sldn_pmn_m))/len(tmp_sldn_pmn_m)*100), label='pmn-m', color='red', linewidth=3)
    plt.plot(np.sort(tmp_sldn_mlsys), (np.arange(len(tmp_sldn_mlsys))/len(tmp_sldn_mlsys)*100), label='mlsys', color='green', linewidth=3)
    # plt.plot(np.sort(tmp_sldn_flowsim), (np.arange(len(tmp_sldn_flowsim))/len(tmp_sldn_flowsim)*100), label='flowsim', color='orange', linewidth=3)
    plt.axhline(99, color='green', linewidth=0.5)
    # plt.xscale('log')
    plt.ylim(80, 100)
    plt.xlabel('slow-down', fontsize=_fontsize)
    plt.yticks(fontsize=_fontsize)
    plt.xticks(fontsize=_fontsize)
    plt.legend()
plt.tight_layout()